In [1]:
import tifffile as tifff
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import matplotlib as mpl
import os

In [2]:
features2fileband = {
    "NDVI": ("median_indexex_2016-2019.tif", 0),
    "NDII": ("median_indexex_2016-2019.tif", 1),
    "GNDVI": ("median_indexex_2016-2019.tif", 2),
    "EVI": ("median_indexex_2016-2019.tif", 3),
    "CHM": ("chm_fvg_10m.tif", 0),
    "PREC_summer": ("pr_avg_rcp45_MAM_ts19762100_ls.tif_2019_2015_interpolated_10m.tif", 0),
    "PREC_spring": ("pr_avg_rcp45_JJA_ts19762100_ls.tif_2019_2015_interpolated_10m.tif", 0),
    "TEMP_summer": ("tas_avg_rcp45_MAM_ts19762100_ls.tif_2019_2015_10m.tif", 0),
    "TEMP_spring": ("tas_avg_rcp45_JJA_ts19762100_ls.tif_2019_2015_10m.tif", 0),
    "ELE": ("DTM_10.tif", 0),
    "SLO": ("slope_percentage_10.tif", 0),
    "ASP": ("aspect_degree_10.tif", 0),
    
    "SAT_1_ultrablue": ("bands_2019-2021.tif",  0),
    "SAT_2_blue": ("bands_2019-2021.tif",  1),
    "SAT_3_green": ("bands_2019-2021.tif",  2),
    "SAT_4_red": ("bands_2019-2021.tif",  3),
    "SAT_5_nir": ("bands_2019-2021.tif",  4),
    "SAT_6_nir": ("bands_2019-2021.tif",  5),
    "SAT_7_nir": ("bands_2019-2021.tif",  6),
    "SAT_8_nir": ("bands_2019-2021.tif",  7),
    "SAT_8A_nir": ("bands_2019-2021.tif",  8),
    "SAT_11_swir": ("bands_2019-2021.tif",  9),
    "SAT_12_swir": ("bands_2019-2021.tif", 10),
}

In [3]:
# black image where inventory points are white pixels
path = "@image_data/punti.tif"
image = tifff.imread(path).astype(int)
# information about the inventory points (private)
df = pd.read_csv("@id_infc.csv")

In [4]:
image.shape

(12078, 12349)

In [5]:
nonzero_values = image[np.where(image>0)]
nonzero_y = np.where(image>0)[0]
nonzero_x = np.where(image>0)[1]

df["center_x"] = df.id_infc.apply(lambda x: nonzero_x[nonzero_values==x][0])
df["center_y"] = df.id_infc.apply(lambda x: nonzero_y[nonzero_values==x][0])

df["xlim_32"]  = df.center_x.apply(lambda x: (x-32//2, x+32//2))
df["xlim_64"]  = df.center_x.apply(lambda x: (x-64//2, x+64//2))
df["xlim_128"] = df.center_x.apply(lambda x: (x-128//2, x+128//2))

df["ylim_32"]  = df.center_y.apply(lambda x: (x-32//2, x+32//2))
df["ylim_64"]  = df.center_y.apply(lambda x: (x-64//2, x+64//2))
df["ylim_128"] = df.center_y.apply(lambda x: (x-128//2, x+128//2))

df.drop(columns=["id_infc", "center_x", "center_y"])

,xlim_32,xlim_64,xlim_128,ylim_32,ylim_64,ylim_128
0,"(3435, 3467)","(3419, 3483)","(3387, 3515)","(386, 418)","(370, 434)","(338, 466)"
1,"(5164, 5196)","(5148, 5212)","(5116, 5244)","(661, 693)","(645, 709)","(613, 741)"
2,"(5048, 5080)","(5032, 5096)","(5000, 5128)","(797, 829)","(781, 845)","(749, 877)"
3,"(5233, 5265)","(5217, 5281)","(5185, 5313)","(832, 864)","(816, 880)","(784, 912)"
4,"(6427, 6459)","(6411, 6475)","(6379, 6507)","(773, 805)","(757, 821)","(725, 853)"
...,...,...,...,...,...,...
274,"(11111, 11143)","(11095, 11159)","(11063, 11191)","(10878, 10910)","(10862, 10926)","(10830, 10958)"
275,"(5885, 5917)","(5869, 5933)","(5837, 5965)","(11022, 11054)","(11006, 11070)","(10974, 11102)"
276,"(5912, 5944)","(5896, 5960)","(5864, 5992)","(11133, 11165)","(11117, 11181)","(11085, 11213)"
277,"(11831, 11863)","(11815, 11879)","(11783, 11911)","(11216, 11248)","(11200, 11264)","(11168, 11296)"


In [9]:
del image

In [10]:
for feature, (path, band) in features2fileband.items():
    
    if not os.path.exists(f"@image_data/{feature}.pkl"):
    
        img = tifff.imread(f"@image_data/{path}")
        if len(img.shape)>2:
            img = img[..., band]
        tmp_df = pd.DataFrame(index=df.index)
        tmp_df[feature+"_32"] = df.apply(lambda row: img[row.ylim_32[0]:row.ylim_32[1], row.xlim_32[0]:row.xlim_32[1]], axis=1)
        tmp_df[feature+"_64"] = df.apply(lambda row: img[row.ylim_64[0]:row.ylim_64[1], row.xlim_64[0]:row.xlim_64[1]], axis=1)
        tmp_df[feature+"_128"] = df.apply(lambda row: img[row.ylim_128[0]:row.ylim_128[1], row.xlim_128[0]:row.xlim_128[1]], axis=1)
        tmp_df.to_pickle(f"@image_data/{feature}.pkl")
        del img
        del tmp_df

In [12]:
for dim in [32, 64, 128]:
    dfs=[]
    for feature in list(features2fileband.keys()):
        tmp = pd.read_pickle(f"@image_data/{feature}.pkl")[[f"{feature}_{dim}"]]
        tmp.columns=[feature]
        dfs.append(tmp)
    df = pd.concat(dfs, axis=1)



    df = df.apply(
        lambda row: pd.concat(
            [
                pd.DataFrame(row[col].tolist(), columns=[col]*dim, index=[row.name]*dim)
                for col in row.index
            ],
            axis=1
        ),
        axis=1
    )


    df = pd.concat(df.tolist())
    df.to_pickle(f"@image_data/{dim}x{dim}.pkl")
    df.to_csv(f"@image_data/{dim}x{dim}.csv")
    display(df)

,NDVI,NDVI,NDVI,NDVI,NDVI,NDVI,NDVI,NDVI,NDVI,NDVI,...,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir
0,0.803525,0.793218,0.795530,0.794945,0.799333,0.669670,0.692508,0.720578,0.581958,0.489332,...,1264.0,1262.5,1220.0,1220.0,1226.0,1226.0,1228.0,1047.0,1047.0,1035.0
0,0.818427,0.811217,0.785927,0.778452,0.778465,0.633114,0.629759,0.634069,0.484917,0.429350,...,1248.0,1212.0,1304.5,1304.5,1372.0,1372.0,1319.0,1246.5,1171.0,1098.0
0,0.824016,0.804485,0.813953,0.786301,0.775330,0.691026,0.674469,0.570767,0.545938,0.513472,...,1176.5,1202.0,1249.0,1298.5,1318.0,1318.0,1238.0,1246.5,1171.0,1098.0
0,0.819101,0.806417,0.815484,0.808724,0.801469,0.738659,0.728272,0.707692,0.666156,0.686615,...,1104.0,1145.0,1197.0,1298.5,1318.0,1318.0,1238.0,1246.5,1171.5,1106.0
0,0.807702,0.811035,0.794551,0.795640,0.770999,0.770999,0.743816,0.744308,0.743429,0.757491,...,1104.0,1145.0,1197.0,1247.0,1252.0,1182.5,1182.5,1065.0,990.5,947.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,0.845726,0.830755,0.822761,0.822761,0.780805,0.813130,0.837557,0.801795,0.801795,0.768794,...,883.5,883.5,957.5,946.0,946.0,946.0,892.0,892.0,828.0,816.0
278,0.840384,0.836642,0.820916,0.820916,0.784673,0.812229,0.828047,0.802796,0.802796,0.775637,...,883.5,883.5,957.5,956.5,956.5,956.5,901.0,901.0,836.0,836.0
278,0.834502,0.836623,0.824665,0.824665,0.807665,0.823235,0.798958,0.802796,0.802796,0.773772,...,883.5,883.5,957.5,956.5,956.5,956.5,901.0,901.0,836.0,836.0
278,0.834502,0.836623,0.824665,0.824665,0.807665,0.823235,0.798958,0.817726,0.817726,0.769014,...,942.5,942.5,1020.5,968.5,968.5,968.5,906.0,906.0,848.0,848.0


,NDVI,NDVI,NDVI,NDVI,NDVI,NDVI,NDVI,NDVI,NDVI,NDVI,...,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir
0,0.803985,0.779297,0.807497,0.819435,0.826556,0.814935,0.851819,0.806675,0.825293,0.839640,...,716.5,716.5,771.0,820.0,1126.0,1126.0,1209.0,1202.5,1203.5,1220.0
0,0.769497,0.769912,0.796403,0.819435,0.826556,0.814935,0.851819,0.848909,0.838695,0.835623,...,731.0,731.0,769.5,806.0,1126.0,1126.0,1209.0,1202.5,1344.5,1309.0
0,0.769497,0.769912,0.796403,0.803753,0.820547,0.833417,0.831205,0.823708,0.827708,0.816658,...,1212.5,1212.5,1214.5,1350.0,1433.5,1433.5,1470.5,1466.5,1418.5,1351.0
0,0.739292,0.756887,0.788983,0.804795,0.849077,0.831806,0.820236,0.811524,0.809628,0.813170,...,1212.5,1212.5,1214.5,1350.0,1433.5,1433.5,1470.5,1466.5,1399.5,1316.0
0,0.784846,0.771593,0.806808,0.822005,0.826303,0.820490,0.821364,0.810337,0.802172,0.799523,...,1245.5,1245.5,1262.0,1323.5,1380.0,1380.0,1377.0,1359.0,1281.0,1250.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,0.833841,0.834183,0.834254,0.840818,0.832819,0.828709,0.839473,0.839473,0.848683,0.843056,...,939.0,1109.0,1109.0,1109.0,997.0,997.0,886.0,886.0,954.0,954.0
278,0.781725,0.829486,0.827342,0.828542,0.827631,0.827046,0.830031,0.830031,0.843876,0.843325,...,939.0,1109.0,1109.0,1109.0,997.0,997.0,886.0,886.0,953.0,953.0
278,0.774392,0.820714,0.830353,0.830721,0.814570,0.796648,0.844322,0.844322,0.847468,0.845855,...,992.0,1059.0,1059.0,1059.0,958.0,958.0,954.0,954.0,1010.0,1010.0
278,0.774820,0.797329,0.802124,0.806876,0.780051,0.769358,0.819561,0.842186,0.840458,0.842652,...,1009.0,1037.0,1037.0,1037.0,951.0,951.0,1026.0,1026.0,1004.0,1004.0


,NDVI,NDVI,NDVI,NDVI,NDVI,NDVI,NDVI,NDVI,NDVI,NDVI,...,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir,SAT_12_swir
0,0.356401,0.298810,0.204972,0.208566,0.592818,0.770386,0.757151,0.774243,0.761959,0.729204,...,667.0,792.0,772.0,720.0,667.0,654.5,674.5,723.0,711.0,631.0
0,0.625662,0.520567,0.411865,0.564425,0.739717,0.791492,0.801743,0.813439,0.790130,0.780009,...,569.0,627.0,602.0,600.5,631.0,620.5,664.0,693.5,711.0,631.0
0,0.685868,0.652051,0.630252,0.710925,0.756005,0.795321,0.813187,0.813769,0.798950,0.797194,...,569.0,629.0,609.5,600.5,631.0,620.5,664.0,693.5,722.0,578.0
0,0.707065,0.717746,0.726148,0.736161,0.756149,0.764200,0.818830,0.828033,0.819440,0.797194,...,569.0,629.0,609.5,605.5,577.0,580.0,629.5,677.0,677.0,582.5
0,0.697954,0.707466,0.724435,0.746856,0.769189,0.764200,0.818830,0.828033,0.819440,0.817951,...,483.0,505.5,506.0,546.5,544.0,547.5,589.5,643.5,637.0,538.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,0.797607,0.775177,0.775177,0.778322,0.785764,0.807012,0.828018,0.828018,0.836066,0.838836,...,857.0,857.0,895.5,895.5,894.0,898.5,905.0,905.0,909.5,917.0
278,0.797401,0.783188,0.783188,0.782670,0.786986,0.837045,0.853980,0.853980,0.831044,0.809370,...,820.0,820.0,895.5,895.5,894.0,898.5,905.0,905.0,909.5,917.0
278,0.790870,0.783188,0.783188,0.782670,0.786986,0.837045,0.853980,0.853980,0.831044,0.809370,...,820.0,820.0,874.0,874.0,908.0,905.0,905.0,909.5,909.5,917.0
278,0.790870,0.777097,0.777097,0.775018,0.786942,0.834586,0.843968,0.843968,0.824940,0.810520,...,820.0,820.0,874.0,874.0,908.0,908.0,932.0,931.5,931.5,932.0
